In [1]:
import json

import re
import scipy
import numpy as np

import scipy.sparse

from tqdm import tqdm_notebook as tqdm

In [2]:
enum_role = "环节"
max_seq_len = 300

In [3]:
def data_process(dataset):
    """data_process"""

    def label_data(data, start, l, _type):
        """label_data"""
        for i in range(start, start + l):
            suffix = "B-" if i == start else "I-"
            if isinstance(data[i], str):
                data[i] = []
            solt = "{}{}".format(suffix, _type)
            if solt not in data[i]:
                data[i].append(solt)  
#             data[i] = "{}{}".format(suffix, _type)
        return data

    output = []
    for d_json in dataset:
        _id = d_json["id"]
        text_a = [
            "，" if t == " " or t == "\n" or t == "\t" else t
            for t in list(d_json["text"].lower())
        ]
        labels = ["O"] * len(text_a)
        if len(d_json.get("event_list", [])) == 0:
            continue
        for event in d_json.get("event_list", []):
            event_type = event["event_type"]
            start = event["trigger_start_index"]
            trigger = event["trigger"]
            labels = label_data(labels, start, len(trigger), event_type)
        output.append({
            "tokens": text_a, "labels": labels
        })
    return output

In [4]:
with open('./resources/duee_fin_train_preprocess.json', 'r', encoding='utf-8') as f:
    dataset = json.loads(f.read())
    preprocess_dataset = data_process(dataset)
    count = 0
    for i, d_json in enumerate(preprocess_dataset):
        if any([len(label) > 1 for label in d_json['labels'] if isinstance(label, list)]):
            print(i, d_json)
            count += 1

55 {'tokens': ['挖', '贝', '网', '8', '月', '6', '日', '，', '梦', '网', '集', '团', '（', '0', '0', '2', '1', '2', '3', '）', '今', '日', '发', '布', '，', '其', '于', '近', '日', '收', '到', '公', '司', '控', '股', '股', '东', '余', '文', '胜', '先', '生', '关', '于', '其', '所', '持', '有', '的', '公', '司', '部', '分', '股', '份', '解', '除', '质', '押', '的', '通', '知', '，', '解', '除', '质', '押', '1', '8', ',', '3', '8', '0', ',', '0', '0', '0', '股', '，', '本', '次', '解', '除', '质', '押', '占', '其', '所', '持', '股', '份', '，', '比', '例', '1', '1', '.', '2', '1', '%', '。', '，', '截', '至', '本', '公', '告', '披', '露', '日', '，', '余', '文', '胜', '先', '生', '持', '有', '公', '司', '股', '份', '1', '6', '3', ',', '9', '7', '8', ',', '3', '7', '5', '股', '，', '占', '公', '司', '总', '股', '本', '的', '2', '0', '.', '2', '3', '%', '，', '其', '所', '持', '有', '公', '司', '股', '份', '累', '计', '被', '质', '押', '1', '1', '7', ',', '4', '4', '0', ',', '8', '5', '5', '股', '，', '占', '其', '所', '持', '股', '份', '总', '数', '的', '7', '1', '.', '6', '2', '%', '，', '占', '公', '司', '总', '股', '本', 

6866 {'tokens': ['原', '标', '题', '：', '烟', '台', '瑞', '康', '医', '药', '(', '7', '.', '5', '6', '0', ',', '-', '0', '.', '0', '3', ',', '-', '0', '.', '4', '0', '%', ')', '实', '控', '人', '7', '.', '6', '9', '%', '股', '份', '解', '除', '质', '押', '，', '，', '来', '源', '：', '鲁', '网', '，', '鲁', '网', '9', '月', '3', '日', '于', '深', '圳', '证', '券', '交', '易', '所', '获', '悉', '，', '瑞', '康', '医', '药', '集', '团', '股', '份', '有', '限', '公', '司', '（', '瑞', '康', '医', '药', '.', '0', '0', '2', '5', '8', '9', '.', 's', 'z', '）', '实', '际', '控', '制', '人', '张', '仁', '华', '质', '押', '给', '中', '泰', '证', '券', '股', '份', '有', '限', '公', '司', '（', '简', '称', '“', '中', '泰', '证', '券', '”', '）', '的', '合', '计', '2', '2', '1', '5', '万', '股', '股', '票', '于', '8', '月', '2', '9', '日', '解', '除', '质', '押', '。', '，', '公', '告', '显', '示', '，', '截', '止', '公', '告', '日', '，', '瑞', '康', '医', '药', '实', '际', '控', '制', '人', '张', '仁', '华', '共', '持', '有', '公', '司', '股', '份', '2', '8', '7', '8', '6', '.', '6', '0', '5', '9', '万', '股', '，', '占', '公', '司'

In [5]:
print(count)
print(len(dataset))

175
28642


In [13]:
with open('./resources/duee_fin_dev_preprocess.json', 'r', encoding='utf-8') as f:
    dataset = json.loads(f.read())
    preprocess_dev_dataset = data_process(dataset)
    count = 0
    for i, d_json in enumerate(preprocess_dev_dataset):
        if any([len(label) > 1 for label in d_json['labels'] if isinstance(label, list)]):
            print(i, d_json)
            count += 1

17 {'tokens': ['每', '经', 'a', 'i', '快', '讯', '，', '方', '盛', '制', '药', '（', 's', 'h', '，', '6', '0', '3', '9', '9', '8', '，', '收', '盘', '价', '：', '6', '.', '8', '7', '元', '）', '7', '月', '7', '日', '晚', '间', '发', '布', '公', '告', '称', '，', '公', '司', '控', '股', '股', '东', '暨', '实', '际', '控', '制', '人', '张', '庆', '华', '先', '生', '于', '7', '月', '7', '日', '将', '约', '1', '7', '7', '9', '.', '6', '5', '万', '股', '解', '除', '质', '押', '。', '，', '同', '时', '，', '张', '庆', '华', '与', '长', '沙', '银', '行', '股', '份', '有', '限', '公', '司', '高', '信', '支', '行', '办', '理', '完', '成', '了', '股', '份', '质', '押', '手', '续', '，', '将', '其', '持', '，', '有', '的', '1', '9', '9', '0', '万', '股', '进', '行', '了', '质', '押', '。', '，', '截', '至', '本', '公', '告', '披', '露', '日', '，', '张', '庆', '华', '先', '生', '直', '接', '持', '有', '公', '司', '的', '股', '份', '总', '数', '约', '为', '1', '.', '5', '6', '亿', '股', '，', '占', '公', '司', '股', '份', '总', '数', '的', '3', '5', '.', '9', '6', '%', '；', '张', '庆', '华', '先', '生', '质', '押', '的', '公', '司', '股', '份', '总', 

In [14]:
print(count)
print(len(dataset))

31
4647


In [8]:
def load_dict(dict_path):
    """load_dict"""
    vocab = {}
    for line in open(dict_path, 'r', encoding='utf-8'):
        value, key = line.strip('\n').split('\t')
        vocab[key] = int(value)
    return vocab

In [9]:
label_vocab = load_dict(dict_path='./dictionary/trigger_tag.dict')

In [10]:
def cal_cover_overlap_conflict(preprocess_dataset):
    coverage_score, overlap_score, conflict_score = 0.0, 0.0, 0.0
    for d_json in tqdm(preprocess_dataset, total=len(preprocess_dataset)):
        labels = d_json['labels']
        label_array = [-1 * np.ones(len(labels))]
        for i, label in enumerate(labels):
            if isinstance(label, str):
                label_array[0][i] = 120
            if isinstance(label, list):
                if len(label_array) < len(label):
                    for j in range(len(label)-len(label_array)):
                        label_array.append(120 * np.ones(len(labels)))
                for j in range(len(label)):
                    l = re.sub(r"\(.*\)", "", label[j])
                    label_array[j][i] = label_vocab.get(l, 120)
        L = np.array(label_array).T
        L_sparse = scipy.sparse.csr_matrix(L + 1)
        covered_data_points = np.ravel(np.where(L_sparse.sum(axis=1) != 0, 1, 0))
        coverage_score += covered_data_points.sum() / L_sparse.shape[0]
        overlapped_data_points = np.where(np.ravel((L_sparse != 0).sum(axis=1)) > 1, 1, 0)
        overlap_score += overlapped_data_points.sum() / L_sparse.shape[0]
        m = scipy.sparse.diags(np.ravel(L_sparse.max(axis=1).todense()))
        conflicted_data_points = np.ravel(np.max(m @ (L_sparse != 0) != L_sparse, axis=1).astype(int).todense())
        conflict_score += conflicted_data_points.sum() / L_sparse.shape[0]
        
    print(f"coverage_score: {coverage_score / len(preprocess_dataset)}, overlap_score: {overlap_score / len(preprocess_dataset)}, conflict_score: {conflict_score / len(preprocess_dataset)}")

In [11]:
cal_cover_overlap_conflict(preprocess_dataset)


coverage_score: 1.0, overlap_score: 0.02413793103448276, conflict_score: 0.00045679124188924785


In [12]:
cal_cover_overlap_conflict(preprocess_dev_dataset)


coverage_score: 1.0, overlap_score: 0.026293469041560644, conflict_score: 0.0004698511377148311
